In [1]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import os, sys, importlib, copy
from glob import glob
import pickle
import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import scipy

from utilities import reduced_parameter as red
from field.electromagnetic import FreeVectorPotential, CavityVectorPotential

from matter.atoms import AtomsInBox
from calculator.calculator import Calculator

from utilities.etc import categorizing_result

import animation as an
import analyze_tools.utils as atu
import analyze_tools.monte_carlo as atm

from simulation.single import single_collision_simulation
import utilities.reduced_parameter as red
from utilities.etc import moving_average


In [2]:
#
# DEFINING FUNCTION
## WARNING: NEXT CELL WILL RUN SIMULATIONS
#


Lxy = 3e7; Lz = 3e3
def get_ArXePair(d_arxe, v_ar = 0, h = 0, n_pairs = 1, Lxy=Lxy, Lz=Lz):
    """
    """
    atoms = AtomsInBox(
        Lxy = Lxy, Lz = Lz, cell_width = (1e6,1e2), 
        mass_dict = red.mass_dict)

    v_xe = - v_ar / np.sqrt(red.mass_dict['Xe'])

    for i in range(n_pairs):
        if i == 0:
            r0 = np.zeros(3)
        else:
            r0 = np.hstack(
                [
                    #np.random.uniform(-Lxy/2+10, Lxy/2-10, size = 2), 
                    np.zeros(2),
                    np.random.uniform(-Lz/2+10, Lz/2-10)]
            )
        atoms.add(
                elements = ["Ar"],
                r = np.array([[0.0, h, -d_arxe/2]]) + r0,
                r_dot = np.array([[0.0, 0.0, v_ar]])# / np.sqrt(2)
                )
        
        atoms.add(
                elements = ["Xe"],
                r = np.array([[0.0,0.0,d_arxe/2]]) + r0,
                r_dot = np.array([[0.00, 0.00, v_xe]])# / np.sqrt(2)
                )
    
    idxAr = atoms.element_idx(element = "Ar")
    idxXe = atoms.element_idx(element = "Xe")
    epsilon_mat, sigma_mat = red.generate_LJparam_matrix(idxAr = idxAr, idxXe = idxXe)
    
    # calculator to the atoms object
    atoms.add_calculator(
        calculator_class = Calculator,
        calculator_kwargs = {
            "epsilon": epsilon_mat, "sigma" : sigma_mat, 
            "positive_atom_idx" : idxXe, "negative_atom_idx" : idxAr,
            "mu0" : red.mu0, "d" : red.d0, "a" : red.a, 'd7':red.d7
        })
    
    atoms.update_distance()
    
    return atoms

def get_field(coup_str, mode = False, Lxy = Lxy, Lz = Lz, field = 'cavity'):
    if mode == 'multi':
        k_vector_int = np.array(
            [[i,0,0] for i in range(1,100)]
            ,dtype=np.float64)
    elif isinstance(mode, int):
        k_vector_int = np.array([[mode,0,0]])
    elif isinstance(mode, list):
        k_vector_int = np.array(
            [[i,0,0] for i in mode], 
            dtype = np.float64)
    else:
        raise Exception('Mode have to be either "multi" or an integer!')
    
    amplitude = np.vstack([
        np.zeros(2) for i in range(len(k_vector_int))
        ])

    if field == 'cavity':
        VectorPotentialClass = CavityVectorPotential
    elif field == 'free':
        VectorPotentialClass = FreeVectorPotential
    
    Afield = VectorPotentialClass(
            k_vector_int = k_vector_int, amplitude = amplitude,
            Lxy = Lxy, Lz = Lz, constant_c = red.c, coupling_strength = coup_str
            )

    return Afield

def get_radprofile(rd, windows = 10):
    Afield = rd['field']
    
    rad_energy = red.convert_energy(np.array(Afield.history["energy"][-1]), "ev") 
    omega = red.convert_wavenumber(Afield.k_val)
    omega_profile, final_rad_profile = at.profiling_rad(omega, rad_energy)
    
    sorted_idx = np.argsort(omega_profile)

    if windows == 0:
        omega_profile = np.array(omega_profile)[sorted_idx] 
        final_rad_profile = np.array(final_rad_profile)[sorted_idx]
    else:
        omega_profile, final_rad_profile = moving_average(
            np.array(omega_profile)[sorted_idx], 
            np.array(final_rad_profile)[sorted_idx], windows)

    return omega_profile, final_rad_profile

In [ ]:
# 
# CALCULATING SPECTRA
# Run simulation of head on collision coupled to FREE field 
# at various total energy as temperature 
# 

result_dict = {}
velocity_dict = {}

for T in tqdm.tqdm(np.arange(92, 850, 10)):
    
    vt = np.sqrt((T * red.boltzmann / red.epsilon) )
    atoms = get_ArXePair(d_arxe = 6, h = 0.0 , v_ar = vt, n_pairs = 1)
    
    field = get_field(mode = list(range(1,300)), coup_str = 1, Lz = Lxy, field = 'free')
        
    t, rd = single_collision_simulation(
        0, h = 1e-2, atoms = atoms, field = field, 
        max_steps = 10000, verbose = False)

    result_dict.update({T : rd})
    velocity_dict.update({T : (vt, vt / np.sqrt(M_Xe))})

In [ ]:
#
# PLOTTING SPECTRA
#

fig,ax = plt.subplots(2, figsize = (6,8))

total = 0

prob_list = []

for T in result_dict.keys():
    prob_list.append(T * np.exp(- T / 292))

prob_list = np.array(prob_list)
prob_list = prob_list / np.sum(prob_list)

for i, (T,rd) in enumerate(result_dict.items()):
    
    omega_profile, final_rad_profile = get_radprofile(rd,0 )
    final_rad_profile *= 1e25
    total += final_rad_profile * prob_list[i]

    if i % 5 == 0:
        ax[0].plot(
            omega_profile, final_rad_profile, label = 'T = {} K'.format(T), 
            linestyle = 'dashdot' if i >= 35 else None)

ax[1].plot(omega_profile, total, label = 'Scaled total spectrum')
  
tax = ax[1].twinx()
exp_data = pd.read_csv('data/plot-data.csv')
tax.scatter(
    exp_data.loc[:,'x'],
    exp_data.loc[:,' y'],
    s = 5, alpha = 0.5, c = 'black'
)
ax[1].scatter([],[], c = 'black', label = 'Exp. @ 292K')

ax[0].set_ylabel(r"Calculated EM field energy $(10^{-25} eV)$")
ax[1].set_ylabel(r"Sum of scaled EM field energy $(10^{-25} eV)$")
ax[1].set_xlabel(r"Wavenumber $(cm^{-1})$")

tax.set_ylabel(r'Exp. absorption $(10^{-6} cm^{-1} amagat^{-2})$')

ax[0].annotate('(a)',xy = (0.9,0.9), xycoords = 'axes fraction')
ax[1].annotate('(b)',xy = (0.9,0.7), xycoords = 'axes fraction')

ax[0].legend(loc='lower center', bbox_to_anchor=(0.5, 1.0), ncol=4)
ax[1].legend()
fig.savefig('figure/manuscript/free_spectrum_singleT.jpeg',dpi = 600, bbox_inches = 'tight')

In [3]:
#
# Record (trajectory @ 40K, highest temperature that Ar-Xe complex form, previous temperature)
#

single_collision_multimode_data = {}

prompt = input('Load file?(Y)/N')
if prompt != 'N':
    raise Exception
    
iplist = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
Tlist = list(range(1, 50, 2)) + [60, 100, 150, 200]

for ct in [15, 1e1, 1]:
    foo = {}
    for h in iplist:
        print('h = ', h, ' coupling strength = ', ct)
        T_break = None

        result_dict = {}
        
        for T in tqdm.tqdm(Tlist[::-1]):

            if ct == 1 and T < 100: break
            
            vt = np.sqrt((T * red.boltzmann / red.epsilon) )
            atoms = get_ArXePair(d_arxe = 6, h = h , v_ar = vt, n_pairs = 1)
            
            field = get_field(
                mode = list(range(1,100)), coup_str = ct * Lxy, Lz = Lz, field = 'cavity')
                
            t, rd = single_collision_simulation(
                0, h = 1e-2, atoms = atoms, field = field, potential_threshold = 1e-4,
                max_steps = 10000, verbose = False, record_every = 1)
            
            collide_time = atm.get_colliding_time(rd['atoms'], 0)[0]
            
            if T == 200: 
                result_dict.update({'200': copy.deepcopy(rd)})
            if T == 150: 
                result_dict.update({'150': copy.deepcopy(rd)})
            if T == 100: 
                result_dict.update({'100': copy.deepcopy(rd)})
            if T == 60: 
                result_dict.update({'60': copy.deepcopy(rd)})
            
            if collide_time > 10:
                result_dict.update(
                    {'last':copy.deepcopy(rd)}
                )
                T_break = T
                print('collide time = ', collide_time, ' break @ ', T)
                break

        foo.update({h : (result_dict, T_break)})
        
    single_collision_multimode_data.update({ct : foo})

Load file?(Y)/N N


h =  0.0  coupling strength =  15


 14%|███████████████████████▏                                                                                                                                                | 4/29 [00:35<03:44,  8.97s/it]


collide time =  206.9107574455581  break @  49
h =  0.2  coupling strength =  15


 14%|███████████████████████▏                                                                                                                                                | 4/29 [00:34<03:38,  8.74s/it]


...

 10%|█████████████████▍                                                                                                                                                      | 3/29 [00:05<00:43,  1.68s/it]


h =  1.0  coupling strength =  1


 10%|█████████████████▍                                                                                                                                                      | 3/29 [00:05<00:45,  1.75s/it]


h =  1.2  coupling strength =  1


 10%|█████████████████▍                                                                                                                                                      | 3/29 [00:05<00:45,  1.76s/it]


In [4]:
prompt = input('Are you sure you want to overwrite existing data? Y/(N)')
if prompt != 'Y': raise Exception
    
prompt = input('Do you have any doubt that you want to overwrite existing data? (Y)/N')
if prompt != 'N': raise Exception

with open('pickle_jar/single_collision/multimode_time_vs_energy.pkl','wb') as handle:
    pickle.dump(single_collision_multimode_data, handle)

Are you sure you want to overwrite existing data? Y/(N) Y
Do you have any doubt that you want to overwrite existing data? (Y)/N N


In [ ]:
with open('pickle_jar/single_collision/multimode_time_vs_energy.pkl','rb') as handle:
    single_collision_multimode_data = pickle.load(handle)

In [5]:
print('coupling strength & h & total kinetic energy')
for i, (ct, dict1) in enumerate(single_collision_multimode_data.items()):
    for j, (h, data) in enumerate(dict1.items()):
        
        print("{} & {} & {} \\\\ ".format(ct, h, data[-1]))

coupling strength & h & total kinetic energy
15 & 0.0 & 49 \\ 
15 & 0.2 & 49 \\ 
15 & 0.4 & 43 \\ 
15 & 0.6 & 37 \\ 
15 & 0.8 & 31 \\ 
15 & 1.0 & 25 \\ 
15 & 1.2 & 19 \\ 
10.0 & 0.0 & 19 \\ 
10.0 & 0.2 & 19 \\ 
10.0 & 0.4 & 19 \\ 
10.0 & 0.6 & 17 \\ 
10.0 & 0.8 & 15 \\ 
10.0 & 1.0 & 11 \\ 
10.0 & 1.2 & 9 \\ 
1 & 0.0 & None \\ 
1 & 0.2 & None \\ 
1 & 0.4 & None \\ 
1 & 0.6 & None \\ 
1 & 0.8 & None \\ 
1 & 1.0 & None \\ 
1 & 1.2 & None \\ 


In [ ]:
single_collision_multimode_data

In [ ]:
single_collision_multimode_data[ct][h][0]

In [6]:
from numpy.linalg import norm as norm

h = 0.0
ct = 15

dataframe = {'T':[], 'coup_str':[], 'h':[], 'theta_ar':[], 'theta_xe':[]}

for i, T in enumerate([200, 150]):
    for h in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2]:
        for ct in [1, 10, 15]:
            v_ar0, v_xe0 = tuple(
                single_collision_multimode_data[ct][h][0][str(T)]['atoms'].trajectory['r_dot'][0]
            )
            v_ar_1, v_xe_1 = tuple(
                single_collision_multimode_data[ct][h][0][str(T)]['atoms'].trajectory['r_dot'][-1]
            )
            
            
            theta_ar = np.arccos((v_ar0 @ v_ar_1.T) / (norm(v_ar0) * norm(v_ar_1))) * 180 / np.pi
            theta_xe = np.arccos((v_xe0 @ v_xe_1.T) / (norm(v_xe0) * norm(v_xe_1))) * 180 / np.pi

            dataframe['T'].append(150 if i == 0 else 100)
            dataframe['coup_str'].append(ct)
            dataframe['h'].append(h)
            dataframe['theta_ar'].append(theta_ar)
            dataframe['theta_xe'].append(theta_xe)

dataframe = pd.DataFrame(dataframe)
dataframe

,T,coup_str,h,theta_ar,theta_xe
0,150,1,0.0,180.000000,180.000000
1,150,10,0.0,179.999996,179.999995
2,150,15,0.0,179.999990,179.999982
3,150,1,0.2,162.205890,139.492985
4,150,10,0.2,161.682827,133.259160
5,150,15,0.2,160.904901,121.064855
6,150,1,0.4,143.856110,108.666998
7,150,10,0.4,142.901446,102.724637
8,150,15,0.4,141.512286,93.633662
9,150,1,0.6,124.289543,85.219269


In [ ]:
dataframe.to_csv('data/deflection_angle.csv')

In [ ]:
subdf = dataframe.loc[dataframe['h'] == 1.2]
subdf

In [ ]:
atoms

In [ ]:
h = 1.0
coup_str = 15

atoms = single_collision_multimode_data[coup_str][h][0]['last']['atoms']

coltime = np.array(atm.get_colliding_time(atoms, 0))
result_idx = np.argmax(coltime)
print(coltime[result_idx])

field = single_collision_multimode_data[coup_str][h][0]['last']['field']

N_pairs = int(len(atoms.r)/2)

r_ar = np.array(atoms.trajectory['r'])[:,result_idx]
r_xe = np.array(atoms.trajectory['r'])[:,result_idx + N_pairs]

v_ar = np.array(atoms.trajectory['r_dot'])[:,result_idx]
v_xe = np.array(atoms.trajectory['r_dot'])[:,result_idx + N_pairs]

t = np.array(atoms.trajectory['t'])

In [ ]:
from calculator.function import LJ_potential

k = 0.5 * red.mass_dict['Ar'] * np.einsum('ni,ni->n',v_ar,v_ar)\
    + 0.5 * red.mass_dict['Xe'] * np.einsum('ni,ni->n',v_xe,v_xe)
k = red.convert_energy(k,'ev') * 1e3

d_arxe = r_ar - r_xe
d_arxe = np.sqrt(np.einsum('ni,ni->n',d_arxe,d_arxe))
v = LJ_potential(red.sigma_Ar_Xe, red.epsilon_Ar_Xe, d_arxe)
v = red.convert_energy(v,'ev') * 1e3

hem = red.convert_energy(
    np.sum(field.history['energy'],axis = 1), 'ev') * 1e3
tem = field.history['t']

fig,ax = plt.subplots(3,figsize = (10,12))
#ax[0].plot(t[200:], d[200:])
ax[0].plot(t, k)
ax[1].plot(t, v)
ax[2].plot(tem[300:], hem[300:])

#ax[0].set_xlim(t[0],t[-1])
ax[0].set_xlim(t[300],t[-1])
ax[1].set_xlim(t[300],t[-1])
ax[2].set_xlim(tem[300],tem[-1])

ax[0].xaxis.set_major_locator(MultipleLocator(5))
ax[0].xaxis.set_major_formatter('{x:.0f}')

ax[1].xaxis.set_major_locator(MultipleLocator(5))
ax[1].xaxis.set_major_formatter('{x:.0f}')

ax[2].xaxis.set_major_locator(MultipleLocator(5))
ax[2].xaxis.set_major_formatter('{x:.0f}')

#ax[0].set_ylabel(r'Ar-Xe distance (nm)')
ax[0].set_ylabel(r'Kinetic energy ($10^{-3}$ eV)')
ax[1].set_ylabel(r'Potential energy ($10^{-3}$ eV)')
ax[2].set_ylabel(r'Total cavity modes energy ($10^{-3}$ eV)')

ax[2].set_xlabel('Time (ps)')

ax[0].annotate('(a)',xy = (0.9,0.9), xycoords = 'axes fraction')
ax[1].annotate('(b)',xy = (0.9,0.9), xycoords = 'axes fraction')
ax[2].annotate('(c)',xy = (0.9,0.9), xycoords = 'axes fraction')

#fig.savefig(fig_root + 'energy_breakdown.jpeg',dpi=600, bbox_inches = 'tight')

In [ ]:
an = importlib.reload(an)

an.single_anime(atoms, index = 0 , N_pairs = N_pairs, save_path = 'figure2/longest_single_anime')